## Scrap a list of skincare products

In [167]:
import requests
import json
def scrap(page_number):
    url = "https://sephora.p.rapidapi.com/us/products/v2/list"

    querystring = {"categoryId":"cat150006","pageSize":"60","currentPage":page_number}

    headers = {
        "X-RapidAPI-Key": "994387d290msh3a8cdb71942f4e6p1c0978jsn6aa55334a8da",
        "X-RapidAPI-Host": "sephora.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"error scraping page {page_number}")
        return None

/Users/camellia.le1912/Desktop/Skin-Care-Recommender-System/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [168]:
skincare_lists = []
for i in range(30, 35):
    new_list = scrap(i)
    if new_list:
        skincare_lists.append(new_list)
        with open("skincareList3.json", "w") as f:
            json.dump(skincare_lists, f, indent=4)

KeyboardInterrupt: 

In [ ]:
def scrap_detail(skuId, productId):
    url = "https://sephora.p.rapidapi.com/us/products/v2/detail"

    querystring = {"productId":productId,"preferedSku":skuId}

    headers = {
        "X-RapidAPI-Key": "994387d290msh3a8cdb71942f4e6p1c0978jsn6aa55334a8da",
        "X-RapidAPI-Host": "sephora.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"error scraping product {productId}")
        return None

In [ ]:
product_data = []
for i in range(len(skincare_lists)):
    for j in range(len(skincare_lists[i])):
        sku_id = skincare_lists[i]['products'][j]['currentSku']['skuId']
        product_id = skincare_lists[i]['products'][j]['productId']
        new_data = scrap_detail(sku_id, product_id)
        if new_data:
            product_data.append(new_data)
            with open("skincareProductDetail3.json", "w") as f:
                json.dump(product_data, f, indent=4)

In [ ]:
import json
with open("skincareProductDetail_all.json", "r") as f:
    test1 = json.load(f)
    
test1.extend(product_data)


In [44]:
import json
import re
with open("skincareProductDetail_all.json", "r") as f:
    data = json.load(f)

In [45]:
def add_name_type_url_brand(data):
    name  = data['productDetails']['displayName'] if 'productDetails' in data else ''
    type = data['parentCategory']['displayName'] if ('parentCategory' in data) else ''
    brand = data['productDetails']['brand']['displayName'] if 'productDetails' in data else ''
    url =  data['fullSiteProductUrl'] if 'fullSiteProductUrl' in data else ''
    price = data['currentSku']['listPrice'] if 'currentSku' in data else ''

    return name, type, brand, url, price
def proce_des(des):
    clean_description = re.sub(r'<[^>]+>', '', des)
    return clean_description[12:]

In [46]:
# Product Name
product_name = data[10]['productDetails']['displayName']
print(product_name)

# Product Type
product_type = data[10]['parentCategory']['displayName']
print(product_type)

# Brand
brand = data[10]['productDetails']['brand']['displayName']
print(brand)

price = data[0]['currentSku']['listPrice']
print(price)


Superfood Gentle Antioxidant Refillable Cleanser
Face Wash & Cleansers
Youth To The People
$24.00


In [47]:
# first = Skincare Concerns:, second = Formulation
# first = Skin Type:, second: Skincare Concerns
def add_sc_concerns(data, first_word, second_word):
    clean_description = ""
    try:
        if 'productDetails' not in data:
            return ''
        long_description_html = data['productDetails']['longDescription']
        indx = long_description_html.find(first_word)
        index_end = long_description_html[indx:].find('</p>')
        if index_end == -1:
            index_end = long_description_html[indx:].find(second_word)
        clean_description = re.sub(r'<[^>]+>', '', long_description_html[indx:indx+index_end])

        if 'Highlighted' in clean_description:
            hinx = clean_description.find('Highlighted')
            clean_description = clean_description[:hinx]
        if 'Formulation' in clean_description:
            hinx = clean_description.find('Formulation')
            clean_description = clean_description[:hinx]
    except:
        print(f'error product')
    return clean_description[len(first_word) + 1:]

In [48]:
def process_skin(data):
    skinType = [['Normal',0], ['Oily', 0], ['Combination',0], ['Dry',0]]
    for i in range(len(skinType)):
        if skinType[i][0] in data:
            skinType[i][1] = 1
    return skinType[0][1],skinType[1][1],skinType[2][1],skinType[3][1]


In [49]:
def get_des(data):
    return re.sub(r'<[^>]+>', '', data['quickLookDescription']) if 'quickLookDescription' in data else ''

In [50]:
processed_data = []

for d in data:
    new_object = {}
    new_object['product_name'], new_object['product_type'], new_object['brand'], new_object['product_href'], new_object['price'] = add_name_type_url_brand(d)
    new_object['notable_effects'] = add_sc_concerns(d, "Skincare Concerns:", "Formulation")
    new_object['skin_type'] = add_sc_concerns(d, "Skin Type:", "Skincare Concerns")
    if len(new_object['skin_type']) == 0:
        new_object['skin_type'] = "Normal, Dry, Combination, and Oily" 
    new_object['Normal'],new_object['Oily'],new_object['Combination'],new_object['Dry'] = process_skin(new_object['skin_type'])
    new_object['description'] = get_des(d)
    processed_data.append(new_object)

In [51]:
import re
re.sub(r'<[^>]+>', '', data[0]['quickLookDescription'])

'A silky vegan balm that hydrates and soothes dry lips in seconds.'

In [52]:
with open('processed_data.json', 'w') as f:
    json.dump(processed_data, f, indent = 4)

In [53]:
len(processed_data)

938

In [54]:
import json

# Load JSON data
with open('processed_data.json') as f:
    data = json.load(f)

import json
import pandas as pd

# Load JSON data
with open('processed_data.json') as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('processed_data.csv', index=True)

print("JSON data successfully converted to CSV.")


JSON data successfully converted to CSV.
